In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar100
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [5]:
num_classes = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169001437/169001437 [==============================] - 649s 4us/step


In [6]:
# phan chia tap train va val
x_val = x_train[40000:50000,]
x_train = x_train[:40000,]
y_val = y_train[40000:50000]
y_train = y_train[:40000]
print(x_train.shape, y_train.shape)

(40000, 32, 32, 3) (40000, 1)


In [7]:
x_train[1].shape

(32, 32, 3)

In [8]:
import cv2
class DataGenerator(tf.keras.utils.Sequence):
    # generates data for keras
    def __init__(self, images, labels=None, batch_size=64, dim=(224,224), 
                 n_classes=100, shuffle=True):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_classes = n_classes
        self.shuffle = shuffle

        self.indexes = np.arange(self.images.shape[0])   # (0,1,2,3,....40000)
        self.on_epoch_end()

    def on_epoch_end(self):
        # updates indexes sau moi epoch
        # tron tap train sau moi epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    def __len__(self):
        # bieu thi so batches moi epoch
        return self.images.shape[0] // self.batch_size
    def __getitem__(self, index):
        # tao batch data trong position index
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        images = []
        labels = []

        for i in batch_indexes:
            img_i = self.images[i]
            img_i = cv2.resize(img_i, self.dim)
            img_i = (img_i)/ 127 - 1    # dua ve tu (-1,1)

            # tang cuong dl
            images.append(img_i)

            label_i = self.labels[i]
            labels.append(label_i)

        images = np.stack(images)  # batch_size, 244,244,3
        labels = np_utils.to_categorical(labels, self.n_classes)

        return images, labels            

In [10]:
# build model
# lay resnet50 trong keras application
# lay weight trong tap ImageNet, ko lay layer cuoi cung
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
# cho lop globalaverage vao lop cuoi, dau ra la 2048
avg = GlobalMaxPooling2D()(resnet_model.output)
# qua lop dense 512
den1 = Dense(512, activation='relu')(avg)
bn = BatchNormalization()(den1)

# tranh overfiting
drop1 = Dropout(0.4)(bn)

# cho qua lop dense cuoi cung, dung softmax
den2 = Dense(num_classes, activation='softmax')(drop1)

model = tf.keras.Model(inputs= resnet_model.inputs, outputs=[den2])

In [12]:
# khi freeze pretrained-model thi nen de batchnorm tiep tuc suy luan, cac layer khac giu nguyen
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
# warm-up
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bs = 16

data_gen = DataGenerator(x_train, y_train, batch_size=bs, dim=(224,224), n_classes=100, shuffle=True)
val_gen = DataGenerator(x_val, y_val, batch_size=bs, dim=(224,224), n_classes=100, shuffle=False)


In [ ]:
t=time.time()
epoch = 5
history_temp = model.fit(data_gen, validation_data=val_gen, epochs=epoch)
print('Training time: %s' %(time.time() - t))

In [ ]:
test_gen = DataGenerator(x_test, y_test, batch_size=bs, dim=(224,224), n_classes=100, shuffle=False)

In [ ]:
# evaluation tren tap test
model.evaluate(test_gen)    # 76%

In [ ]:
# unfreeze pretrain model, update toan bo model
# unfreeze weights
for layer in resnet_model.layers:
    layer.trainable = True

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
# su dung earlystopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
hist = model.fit(data_gen, validation_data=val_gen, epochs=25, callbacks=[es], verbose=1)

In [ ]:
# save model
model.save_weights